In [39]:
import pandas as pd
import numpy as np
import warnings
import plotly.express as px

warnings.filterwarnings('ignore')
np.random.seed(42)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.2f}'.format

# Trabalhando os Dados Iniciais

Critérios de corte Inicial:
Importações menores que 1000 litros no total de 15 anos.
Países que só fizeram 1 ou 2 compras no período de 15 anos e não foram nos anos de 2021 ou 2022 (Compradores eventuais)
Corte da Rússia que causa uma distorção nos dados por conta de um acordo comercial entre Brasil e Rússia que fez o ano de 2009 ter um volume fora da realidade. Sem contar que a indefinição da guerra com a Ukrânia nao inspira confiança em investir em uma parceria neste momento.

In [40]:
df = pd.read_excel('./ExportacaoVinhos.xlsx')
df_inicial = df[df.columns[0:31]]
df_inicial.head()

,Pais,2008_KG,2008_USD,2009_KG,2009_USD,2010_KG,2010_USD,2011_KG,2011_USD,2012_KG,2012_USD,2013_KG,2013_USD,2014_KG,2014_USD,2015_KG,2015_USD,2016_KG,2016_USD,2017_KG,2017_USD,2018_KG,2018_USD,2019_KG,2019_USD,2020_KG,2020_USD,2021_KG,2021_USD,2022_KG,2022_USD
0,Paraguai,2191901,1374088,486927,392087,510989,449197,240168,276281,354824,428279,481564,680828,521847,908028,495428,741370,985739,1655417,2393468,4274650,3234168,5494321,2419537,3826587,3299013,3869243,6522527,7192362,5076670,7156293
1,Estados Unidos,443895,804607,372319,660066,228968,478630,306787,1030254,146585,303986,245368,786556,222267,494216,195896,524109,258072,687411,132688,1523699,169109,512519,209765,616274,300178,610793,111085,203554,220373,447893
2,China,8689,25926,1553416,482400,795,2358,54156,334867,87905,642177,40929,279956,64040,455340,47609,222866,134106,499622,67594,266086,30835,126336,129852,376828,122253,363000,61884,264116,105395,404647
3,Espanha,2942,6834,2181,4050,0,0,5206,24618,0,0,1972980,3748940,0,0,0,0,0,0,0,0,6123,22631,3540,1353,28,126,0,0,0,0
4,Haiti,20,20,4500,5863,2700,3750,0,0,0,0,0,0,0,0,0,0,0,0,0,0,79500,144425,81873,129803,399128,471152,670379,831181,553503,741014


In [41]:
df_inicial.shape

(71, 31)

In [42]:
df_melted = pd.melt(df_inicial, id_vars=['Pais'], var_name='Ano e Valor', value_name='Quantidade')
df_melted[['Ano', 'Valor']] = df_melted['Ano e Valor'].str.split('_', expand=True)
df_exportacao = df_melted.pivot(index=['Pais', 'Ano'], columns='Valor', values='Quantidade').reset_index()
df_exportacao.columns.name = None 
df_exportacao = df_exportacao.rename(columns={'KG': 'Litros', 'Pais' : 'Destino'})
df_exportacao['Origem'] = 'Brasil'
df_exportacao['Ano'] = pd.to_datetime(df_exportacao['Ano'], format='%Y')
df_exportacao['Litros'] = df_exportacao['Litros'].astype(float)
df_exportacao['USD'] = df_exportacao['USD'].astype(float)
df_exportacao.head()

,Destino,Ano,Litros,USD,Origem
0,Alemanha,2008-01-01,"265,742.00","429,970.00",Brasil
1,Alemanha,2009-01-01,"225,086.00","393,482.00",Brasil
2,Alemanha,2010-01-01,"27,715.00","138,666.00",Brasil
3,Alemanha,2011-01-01,"36,070.00","144,150.00",Brasil
4,Alemanha,2012-01-01,"8,189.00","56,342.00",Brasil


In [43]:
df_exportacao.shape

(1065, 5)

In [44]:
df_exportacao.describe()

,Ano,Litros,USD
count,1065,"1,065.00","1,065.00"
mean,2015-01-01 03:12:00,"45,938.76","81,684.62"
min,2008-01-01 00:00:00,0.00,0.00
25%,2011-01-01 00:00:00,0.00,0.00
50%,2015-01-01 00:00:00,468.00,"1,549.00"
75%,2019-01-01 00:00:00,"7,179.00","23,780.00"
max,2022-01-01 00:00:00,"6,522,527.00","7,192,362.00"
std,NaN,"330,010.49","449,592.23"


In [45]:
df_exportacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Destino  1065 non-null   object        
 1   Ano      1065 non-null   datetime64[ns]
 2   Litros   1065 non-null   float64       
 3   USD      1065 non-null   float64       
 4   Origem   1065 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 41.7+ KB


In [46]:
df_exportacao['Destino'].nunique()

71

#### Anotação: Nos últimos 15 anos nós exportamos vinho para 71 Países.

# Gerando a Tabela Solicitada Pelo Head de Dados

In [47]:
df_total_por_pais = df_exportacao.groupby(['Origem','Destino']).sum(['Litros','USD']).sort_values('Litros', ascending=False)
df_total_por_pais.reset_index(inplace=True)
df_total_por_pais.head()

,Origem,Destino,Litros,USD
0,Brasil,Paraguai,"29,214,770.00","38,719,031.00"
1,Brasil,Estados Unidos,"3,563,355.00","9,684,567.00"
2,Brasil,China,"2,509,458.00","4,746,525.00"
3,Brasil,Espanha,"1,993,000.00","3,808,552.00"
4,Brasil,Haiti,"1,791,603.00","2,327,208.00"


In [48]:
df_total_por_pais.to_excel('./ExportacaoVinhosTotais.xlsx', index=False)

# Trabalhando com os dados totais dos últimos 15 anos

In [49]:
df_total_por_pais['USD_por_Litro'] = df_total_por_pais['USD'] / df_total_por_pais['Litros']
df_total_por_pais.head(10)

,Origem,Destino,Litros,USD,USD_por_Litro
0,Brasil,Paraguai,"29,214,770.00","38,719,031.00",1.33
1,Brasil,Estados Unidos,"3,563,355.00","9,684,567.00",2.72
2,Brasil,China,"2,509,458.00","4,746,525.00",1.89
3,Brasil,Espanha,"1,993,000.00","3,808,552.00",1.91
4,Brasil,Haiti,"1,791,603.00","2,327,208.00",1.30
5,Brasil,Reino Unido,"1,239,551.00","4,711,464.00",3.80
6,Brasil,Paises Baixos,"1,236,154.00","3,791,611.00",3.07
7,Brasil,Japão,"1,181,692.00","2,377,716.00",2.01
8,Brasil,Alemanha,"909,051.00","2,546,394.00",2.80
9,Brasil,Uruguai,"792,595.00","1,219,878.00",1.54


### Optei por trabalhar com os dados dos top 10 em volume de exportação que representam 90,82% dos nossos dados.

In [94]:
top10 = df_total_por_pais.sort_values('Litros', ascending=False)['Destino'].head(10).to_list()
top10

['Paraguai',
 'Estados Unidos',
 'China',
 'Espanha',
 'Haiti',
 'Reino Unido',
 'Paises Baixos',
 'Japão',
 'Alemanha',
 'Uruguai']

In [95]:
df_total_top10 = df_total_por_pais[df_total_por_pais['Destino'].isin(top10)]
df_total_top10

,Origem,Destino,Litros,USD,USD_por_Litro
0,Brasil,Paraguai,"29,214,770.00","38,719,031.00",1.33
1,Brasil,Estados Unidos,"3,563,355.00","9,684,567.00",2.72
2,Brasil,China,"2,509,458.00","4,746,525.00",1.89
3,Brasil,Espanha,"1,993,000.00","3,808,552.00",1.91
4,Brasil,Haiti,"1,791,603.00","2,327,208.00",1.30
5,Brasil,Reino Unido,"1,239,551.00","4,711,464.00",3.80
6,Brasil,Paises Baixos,"1,236,154.00","3,791,611.00",3.07
7,Brasil,Japão,"1,181,692.00","2,377,716.00",2.01
8,Brasil,Alemanha,"909,051.00","2,546,394.00",2.80
9,Brasil,Uruguai,"792,595.00","1,219,878.00",1.54


In [100]:
fig = px.bar(df_total_top10, x='Destino', y=['Litros'], title='Litros exportados destino')
fig.show()

In [102]:
fig = px.bar(df_total_top10, x='Destino', y=['USD'], title='Dolares por destino')
fig.show()

In [97]:
fig = px.bar(df_total_top10.sort_values(by='USD_por_Litro',ascending=False), x='Destino', y=['USD_por_Litro'], title='Dolar por Litro em cada destino')
fig.show()

In [98]:
df_producao_exportacao = pd.read_excel('./ProducaoExportacao.xlsx')
df_producao_exportacao['Ano'] = pd.to_datetime(df_producao_exportacao['Ano'], format='%Y')
df_producao_exportacao['Produzido'] = df_producao_exportacao['Produzido'].astype(float)
df_producao_exportacao['Exportado'] = df_producao_exportacao['Exportado'].astype(float)
df_producao_exportacao.head()

,Ano,Produzido,Exportado
0,2008-01-01,"334,841,312.00","10,346,323.00"
1,2009-01-01,"245,318,774.00","25,514,198.00"
2,2010-01-01,"220,073,693.00","1,280,574.00"
3,2011-01-01,"305,439,220.00","1,214,834.00"
4,2012-01-01,"257,977,767.00","5,775,376.00"


In [104]:
fig = px.bar(df_producao_exportacao.sort_values(by='Produzido',ascending=False), x='Ano', y=['Produzido','Exportado'], title='Produzido vs Exportado', barmode='group')
fig.show()

In [99]:
total_produzido = df_producao_exportacao['Produzido'].sum()
total_exportado = df_producao_exportacao['Exportado'].sum()
percentual_exportado= (total_exportado/total_produzido) * 100

print(f"Total Produzido: {total_produzido}")
print(f"Total Exportado: {total_exportado}")
print(f"Percentual Exportado: {percentual_exportado}")

Total Produzido: 3548338710.0
Total Exportado: 87982432.0
Percentual Exportado: 2.4795387134843168


# Iniciando a Análise anualizada

### Dados Brasil como produtor e exportador

In [106]:
df_producao_exportacao

,Ano,Produzido,Exportado
0,2008-01-01,"334,841,312.00","10,346,323.00"
1,2009-01-01,"245,318,774.00","25,514,198.00"
2,2010-01-01,"220,073,693.00","1,280,574.00"
3,2011-01-01,"305,439,220.00","1,214,834.00"
4,2012-01-01,"257,977,767.00","5,775,376.00"
5,2013-01-01,"242,686,752.00","9,149,059.00"
6,2014-01-01,"234,637,437.00","2,324,029.00"
7,2015-01-01,"247,457,542.00","1,198,479.00"
8,2016-01-01,"104,389,641.00","1,786,549.00"
9,2017-01-01,"299,553,057.00","2,889,572.00"
